In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#pip list
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
!pip install -qq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 59.

In [ ]:
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/kannada_dataset_2_train.csv")
dataset2_filtered = dataset2[dataset2['label'] != 2]
X=list(dataset2_filtered['text'])
y=list(dataset2_filtered['label'])

In [ ]:
dataset2_filtered.shape

(5874, 2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=42,test_size=0.3)
train_df = pd.DataFrame({"text": X_train, "labels": y_train})
test_df = pd.DataFrame({"text": X_test, "labels": y_test})

In [ ]:
train_df.shape

(4111, 2)

In [ ]:
test_df.shape

(1763, 2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=42,test_size=0.3)
train_df = pd.DataFrame({"text": X_train, "labels": y_train})
test_df = pd.DataFrame({"text": X_test, "labels": y_test})

In [ ]:
import torch
torch.cuda.device_count()

1

In [ ]:
cuda_available = torch.cuda.is_available()

# Assign device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

# XLM_RoBERTa

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 15,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 3,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 2e-5,
    'train_batch_size': 64,
    "use_early_stopping": True,
}

xlm_model_kannada = ClassificationModel(
    "xlmroberta",
    "xlm-roberta-base",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:484: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [ ]:
xlm_model_kannada.train_model(train_df)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 3 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 4 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

...

Running Epoch 12 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 15 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

(975, 0.30500553214397186)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = xlm_model_kannada.eval_model(test_df)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
print(result) #result of the model

{'mcc': 0.49079682833575816, 'accuracy': 0.8145207033465683, 'f1_score': 0.6093189964157706, 'tp': 255, 'tn': 1181, 'fp': 133, 'fn': 194, 'auroc': 0.8548372334258778, 'auprc': 0.7140865093936855, 'eval_loss': 0.5915794256660674}


In [ ]:
predictions, raw_outputs = xlm_model_kannada.predict(list(test_df['text'].values))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['labels'].values,zero_division=True))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      1375
           1       0.57      0.66      0.61       388

    accuracy                           0.81      1763
   macro avg       0.73      0.76      0.74      1763
weighted avg       0.83      0.81      0.82      1763



In [ ]:
accuracy = accuracy_score(test_df['labels'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8145207033465683


In [ ]:
f1 = f1_score(test_df['labels'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.8098656999539398


In [ ]:
import pickle
# save the model to disk
filename = 'xlm_roberta_kannada.sav'
pickle.dump(xlm_model_kannada, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
sample_text="guru deshgaande swantha picture tegeyo sule maga alla yaav picture remake idhuthinking face " #2
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
import shutil

local_model_path = '/content/sample_data/xlm_roberta_kannada.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Kannada/xlm_roberta_kannada.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Kannada/xlm_roberta_kannada.sav'

# mBERT

In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 15,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 4,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 3e-5,
    'train_batch_size': 64,
    "use_early_stopping": True,
}

mbert_model_kannada = ClassificationModel(
    "bert",
    "bert-base-multilingual-uncased",  # Use the multilingual model
    num_labels=2,
    args=train_args,
    use_cuda=True
)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
mbert_model_kannada.train_model(train_df)

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 3 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

...

Running Epoch 12 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 15 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

(975, 0.14081399037448977)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = mbert_model_kannada.eval_model(test_df)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
print(result) #result of the model

{'mcc': 0.5055427438349024, 'accuracy': 0.8213272830402722, 'f1_score': 0.6172539489671932, 'tp': 254, 'tn': 1194, 'fp': 120, 'fn': 195, 'auroc': 0.7865542572196629, 'auprc': 0.6558556638081453, 'eval_loss': 1.096048994196786}


In [ ]:
predictions, raw_outputs = mbert_model_kannada.predict(list(test_df['text'].values))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['labels'].values,zero_division=True))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88      1389
           1       0.57      0.68      0.62       374

    accuracy                           0.82      1763
   macro avg       0.74      0.77      0.75      1763
weighted avg       0.84      0.82      0.83      1763



In [ ]:
accuracy = accuracy_score(test_df['labels'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8213272830402722


In [ ]:
f1 = f1_score(test_df['labels'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.8156648708850814


In [ ]:
import pickle
# save the model to disk
filename = 'mbert_kannada.sav'
pickle.dump(mbert_model_kannada, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
sample_text="namma janagalige taste illa avrige fighting " #Our people have no taste for fighting 1
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
sample_text="alla ri valmiki real life na khate bardidaro real khate bardidaro avnu ganeshana hattira barsiridu avaga ramayana mahabharata nadide ella haganta ashvataama badkidane anta hege nambtira astukku adu nadidide elvo namg gotte ella adu hege aahvataama badkidane anta heltira edu bidi nivu avna nodidira bandu kate helda anta ede alla nivu hogi kelidira avaru buildope ge helirbodu edu yarige esta agidiyo avru like madi " #2
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
import shutil

local_model_path = '/content/sample_data/mbert_kannada.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Kannada/mbert_kannada.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Kannada/mbert_kannada.sav'

# DistilBERT



In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 5,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 2e-5,
    'train_batch_size': 64,
    "use_early_stopping": True,
}

distilbert_model_kannada = ClassificationModel(
    "bert",
    "distilbert-base-multilingual-cased",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['encoder.layer.2.attention.output.dense.weight', 'encoder.layer.7.attention.output.dense.bias', 'encoder.layer.8.output.LayerNorm.weight', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.3.output.dense.bias', 'encoder.layer.11.attention.output.dense.weight', 'encoder.layer.4.attention.self.key.bias', 'encoder.layer.5.output.LayerNorm.bias', 'encoder.layer.8.attention.self.value.weight', 'encoder.layer.4.intermediate.dense.weight', 'encoder.layer.5.attention.self.query.weight', 'encoder.layer.1.attention.self.value.weight', 'encoder.layer.11.attention.output.dense.bias', 'encoder.layer.7.attention.self.query.bias', 'encoder.layer.11.attention.output.LayerNorm.bias', 'e

In [ ]:
distilbert_model_kannada.train_model(train_df)

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/65 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/65 [00:00<?, ?it/s]

(1300, 0.25657811475631137)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = distilbert_model_kannada.eval_model(test_df)
print(result)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.4660354473438602, 'accuracy': 0.8077141236528644, 'f1_score': 0.5860805860805862, 'tp': 240, 'tn': 1184, 'fp': 130, 'fn': 209, 'auroc': 0.8272094592075067, 'auprc': 0.6768781890734078, 'eval_loss': 0.6912765304247538}


In [ ]:
predictions, raw_outputs = distilbert_model_kannada.predict(list(test_df['text'].values))
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['labels'].values,zero_division=True))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.90      0.85      0.87      1393
           1       0.53      0.65      0.59       370

    accuracy                           0.81      1763
   macro avg       0.72      0.75      0.73      1763
weighted avg       0.82      0.81      0.81      1763



In [ ]:
accuracy = accuracy_score(test_df['labels'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8077141236528644


In [ ]:
f1 = f1_score(test_df['labels'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.8012460595715162


In [ ]:
import pickle
# save the model to disk
filename = 'distilbert_kannada.sav'
pickle.dump(distilbert_model_kannada, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import shutil

local_model_path = '/content/sample_data/distilbert_kannada.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Kannada/distilbert_kannada.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Kannada/distilbert_kannada.sav'

In [ ]:
sample_text="nanna prakara badukira bhahudu adare media davari aa cotiyalli ashvattama na bage yake gottaglilla " #3
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

# Muril

In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 15,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 2,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 3e-5,
    'train_batch_size': 64,
    "use_early_stopping": True,
}

muril_model_kannada = ClassificationModel(
    "bert",
    "google/muril-base-cased",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
muril_model_kannada.train_model(train_df)

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 3 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

...

Running Epoch 12 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

Running Epoch 15 of 15:   0%|          | 0/65 [00:00<?, ?it/s]

(975, 0.35968992771246494)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = muril_model_kannada.eval_model(test_df)
print(result)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.5199473017016653, 'accuracy': 0.8258650028360749, 'f1_score': 0.6296743063932448, 'tp': 261, 'tn': 1195, 'fp': 119, 'fn': 188, 'auroc': 0.8374774994660891, 'auprc': 0.6265942238737859, 'eval_loss': 0.4923134081893497}


In [ ]:
predictions, raw_outputs = muril_model_kannada.predict(list(test_df['text'].values))
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['labels'].values,zero_division=True))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.91      0.86      0.89      1383
           1       0.58      0.69      0.63       380

    accuracy                           0.83      1763
   macro avg       0.75      0.78      0.76      1763
weighted avg       0.84      0.83      0.83      1763



In [ ]:
accuracy = accuracy_score(test_df['labels'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8258650028360749


In [ ]:
f1 = f1_score(test_df['labels'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.8208456635499957


In [ ]:
import pickle
# save the model to disk
filename = 'muril_kannada.sav'
pickle.dump(muril_model_kannada, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import shutil

local_model_path = '/content/sample_data/muril_kannada.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Kannada/muril_kannada.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Kannada/muril_kannada.sav'

In [ ]:
sample_text="muchgond irappa nim dharmad bagge nodkond sumne irinnam nambike bagge nimag yeke uri " #2
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
sample_text="nanna prakara badukira bhahudu adare media davari aa cotiyalli ashvattama na bage yake gottaglilla " #3
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])